In [2]:
import keras
import numpy as np
import pandas as pd
import matchzoo as mz

Using TensorFlow backend.
Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [3]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [4]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8779.92it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4717.11it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614976/1614976 [00:00<00:00, 4010304.05it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 2118/2118 [00:00<00:00, 6388.85it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 18841/18841 [00:05<00:00, 3540.96it/s]


In [5]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 122/122 [00:00<00:00, 5893.45it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 1115/1115 [00:00<00:00, 3252.33it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 237/237 [00:00<00:00, 5768.80it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 2300/2300 [00:00<00:00, 2831.56it/s]


In [6]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [7]:
model = mz.models.DSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DSSM.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 9644)         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 9644)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          2893500     text_left[0][0]                  
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 300)          2893500     text_right[0][0]                 
_______________________________________________________________________________

In [8]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [11]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=1, num_neg=4, batch_size=64, shuffle=True)
len(train_generator)

16

In [12]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/20
16/16 [==============================] - 3s 167ms/step - loss: 0.9046
Validation: normalized_discounted_cumulative_gain@3(0): 0.4766756532729437 - normalized_discounted_cumulative_gain@5(0): 0.5443368936629973 - mean_average_precision(0): 0.4972338155737947
Epoch 2/20
16/16 [==============================] - 2s 149ms/step - loss: 0.8322
Validation: normalized_discounted_cumulative_gain@3(0): 0.5075174785486662 - normalized_discounted_cumulative_gain@5(0): 0.5746789352776713 - mean_average_precision(0): 0.5251236603293564
Epoch 3/20
16/16 [==============================] - 2s 149ms/step - loss: 0.8010
Validation: normalized_discounted_cumulative_gain@3(0): 0.49573125676219565 - normalized_discounted_cumulative_gain@5(0): 0.5664837338461479 - mean_average_precision(0): 0.5108378267980199
Epoch 4/20
16/16 [==============================] - 2s 156ms/step - loss: 0.7789
Validation: normalized_discounted_cumulative_gain@3(0): 0.507546861485112 - normalized_discounted_cumulative_ga